In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from datetime import datetime

drive.mount('/content/drive')

path = '/content/drive/MyDrive/Popular_Baby_Names.csv'

import pandas as pd
df = pd.read_csv(path)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
0,2011,FEMALE,HISPANIC,GERALDINE,13,75
1,2011,FEMALE,HISPANIC,GIA,21,67
2,2011,FEMALE,HISPANIC,GIANNA,49,42
3,2011,FEMALE,HISPANIC,GISELLE,38,51
4,2011,FEMALE,HISPANIC,GRACE,36,53
...,...,...,...,...,...,...
77282,2012,MALE,BLACK NON HISP,CAYDEN,19,52
77283,2013,FEMALE,WHITE NON HISPANIC,Margaret,25,67
77284,2013,FEMALE,WHITE NON HISPANIC,Tamar,10,82
77285,2013,FEMALE,WHITE NON HISPANIC,Amanda,13,79


In [ ]:
import pandas as pd
import re

# Regex patterns for names
full_name_pattern = re.compile(r'^[A-Z][a-z]+\s[A-Z][a-z]+$', re.IGNORECASE)  # "John Doe"
single_name_pattern = re.compile(r'^[A-Z][a-z]+$', re.IGNORECASE)  # "John" or "Doe"

# Common non-name values (race, gender labels, etc.)
non_name_values = {"male", "female", "nonbinary", "black", "white", "asian", "hispanic", "latino", "native"}

# Function to refine name column detection
def find_name_columns(df):
    name_columns = []

    for column in df.columns:
        values = df[column].astype(str)

        match_count_full = values.apply(lambda x: bool(full_name_pattern.match(x))).sum()
        match_count_single = values.apply(lambda x: bool(single_name_pattern.match(x))).sum()
        non_name_count = values.apply(lambda x: x.lower() in non_name_values).sum()

        # Check if values are consistently short (likely a name, not descriptive text)
        avg_word_count = values.apply(lambda x: len(str(x).split())).mean()

        # Consider a column a "name column" if:
        # - At least 70% of values match name patterns
        # - Less than 10% match race/gender labels
        # - Word count average is **≤ 2** (filters out descriptive categorical data)
        if max(match_count_full, match_count_single) / len(values) > 0.7 and non_name_count / len(values) < 0.1 and avg_word_count <= 2:
            name_columns.append(column)

    return name_columns

# Identify columns containing names
name_columns = find_name_columns(df)

# Print results
if name_columns:
    print(f"✅ Name columns found: {name_columns}")
else:
    print("❌ No name columns detected.")

✅ Name columns found: ["Child's First Name"]


In [ ]:
import pandas as pd


# Function to extract only the first letter of names
def first_letter_only(name):
    return name[0] if isinstance(name, str) and name.strip() != "" else ""  # Ensures valid input

# Apply transformation **only to first 50 rows**
df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)

# Print results
print(df.head(50))  # Show first 50 modified rows

    Year of Birth  Gender Ethnicity Child's First Name  Count  Rank
0            2011  FEMALE  HISPANIC                  G     13    75
1            2011  FEMALE  HISPANIC                  G     21    67
2            2011  FEMALE  HISPANIC                  G     49    42
3            2011  FEMALE  HISPANIC                  G     38    51
4            2011  FEMALE  HISPANIC                  G     36    53
5            2011  FEMALE  HISPANIC                  G     26    62
6            2011  FEMALE  HISPANIC                  H    126     8
7            2011  FEMALE  HISPANIC                  H     14    74
8            2011  FEMALE  HISPANIC                  H     17    71
9            2011  FEMALE  HISPANIC                  H     17    71
10           2011  FEMALE  HISPANIC                  H     13    75
11           2011  FEMALE  HISPANIC                  H     10    78
12           2011  FEMALE  HISPANIC                  H     15    73
13           2011  FEMALE  HISPANIC             

<ipython-input-3-2400b8188af2>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)


In [ ]:
import pandas as pd

# Function to classify individual values
def classify_name(value):
    value = str(value).strip()
    if len(value) == 1:  # Single character = Initial
        return "Initial"
    elif len(value) > 1:  # More than one character = Full Name
        return "Full Name"
    else:
        return "Unknown"


# Apply classification function to each value
for col in name_columns:
    df[f"{col} Type"] = df[col].apply(classify_name)

# Count full names and initials
for col in name_columns:
    full_names_count = (df[f"{col} Type"] == "Full Name").sum()
    initials_count = (df[f"{col} Type"] == "Initial").sum()

    # Print results
    print(f"❌ {col}: {full_names_count} full names, {initials_count} initials, Data is inconsistent. Suggest getting the full name for values with just initials.")

❌ Child's First Name: 77237 full names, 50 initials, Data is inconsistent. Suggest getting the full name for values with just initials.
